# Визуализация

In [1]:
from pathlib import Path
from pprint import pprint

import pandas as pd
import time
import os

import tiktoken

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
# from graphrag.query.llm.oai.chat_openai import ChatOpenAI
# from graphrag.query.llm.oai.embedding import OpenAIEmbedding
# from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

In [4]:
directory = os.path.join(os.getcwd(), "graphragcad")
directory

'C:\\Users\\glvv2\\hr_assistant\\graphragcad'

In [5]:
INPUT_DIR = os.path.join(directory, "output")
LANCEDB_URI = os.path.join(INPUT_DIR, "lancedb")

In [6]:
COMMUNITY_REPORT_TABLE = "community_reports"
ENTITY_TABLE = "entities"
COMMUNITY_TABLE = "communities"
RELATIONSHIP_TABLE = "relationships"
COVARIATE_TABLE = "covariates"
TEXT_UNIT_TABLE = "text_units"
COMMUNITY_LEVEL = 2

### Чтение сущностей (entities)

In [7]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
community_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_TABLE}.parquet")

In [8]:
entity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2706 non-null   object 
 1   human_readable_id  2706 non-null   int64  
 2   title              2706 non-null   object 
 3   type               2706 non-null   object 
 4   description        2706 non-null   object 
 5   text_unit_ids      2706 non-null   object 
 6   frequency          2706 non-null   int64  
 7   degree             2706 non-null   int32  
 8   x                  2556 non-null   float64
 9   y                  2556 non-null   float64
dtypes: float64(2), int32(1), int64(2), object(5)
memory usage: 201.0+ KB


In [10]:
community_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540 entries, 0 to 539
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 540 non-null    object
 1   human_readable_id  540 non-null    int64 
 2   community          540 non-null    int64 
 3   level              540 non-null    int64 
 4   parent             540 non-null    int32 
 5   children           540 non-null    object
 6   title              540 non-null    object
 7   entity_ids         540 non-null    object
 8   relationship_ids   540 non-null    object
 9   text_unit_ids      540 non-null    object
 10  period             540 non-null    object
 11  size               540 non-null    int64 
dtypes: int32(1), int64(4), object(7)
memory usage: 48.6+ KB


### Чтение связей (relationships)

In [9]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

In [13]:
relationships[0]

Relationship(id='4f65e304-2128-49b4-b2f3-80b0dfeaf2fb', short_id='0', source='УПРАВЛЕНИЕ ВИЗУАЛИЗАЦИЕЙ МОДЕЛИ', target='ПАНЕЛЬ «ВИД»', weight=9.0, description='Функции управления визуализацией модели расположены на панели «Вид»', description_embedding=None, text_unit_ids=['cbc5a6c172c48df09239519f9b9f7cff921178a850a50a1ccc014060ae2aca38628eaa31d17bd69958db6f46bc50611a75f9820093b8eb95ec500191aaeea78e'], rank=15, attributes=None)

In [ ]:
relationship_df.info()

## Визуализация с помощью yfiles-jupyter-graphs

In [14]:
from yfiles_jupyter_graphs import GraphWidget

In [15]:
# converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
def convert_entities_to_dicts(df):
    """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
    nodes_dict = {}
    for _, row in df.iterrows():
        # Create a dictionary for each row and collect unique nodes
        node_id = row["title"]
        if node_id not in nodes_dict:
            nodes_dict[node_id] = {
                "id": node_id,
                "properties": row.to_dict(),
            }
    return list(nodes_dict.values())

In [16]:
# converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
def convert_relationships_to_dicts(df):
    """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
    relationships = []
    for _, row in df.iterrows():
        # Create a dictionary for each row
        relationships.append({
            "start": row["source"],
            "end": row["target"],
            "properties": row.to_dict(),
        })
    return relationships

In [17]:
w = GraphWidget()
w.directed = True

In [18]:
entity_df_crop = entity_df.drop(columns=['text_unit_ids'])
entity_df_crop.loc[entity_df['y'].isnull(), 'x'] = 0
entity_df_crop.loc[entity_df['y'].isnull(), 'y'] = 0

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
len(entity_df_crop["type"].unique())

31

In [21]:
enc = LabelEncoder()
temp = enc.fit_transform(entity_df_crop["type"])
temp

array([ 4,  1, 26, ..., 20,  0,  0])

In [22]:
entity_df_crop["type_c"] = temp

In [23]:
w.nodes = convert_entities_to_dicts(entity_df_crop)

In [24]:
relationship_df_crop = relationship_df.drop(columns=['text_unit_ids'])
w.edges = convert_relationships_to_dicts(relationship_df_crop)

## Конфигурация визуализации

In [25]:
def community_to_color(community):
    """Map a community to a color."""
    colors = [
        "crimson",
        "darkorange",
        "indigo",
        "cornflowerblue",
        "cyan",
        "teal",
        "green",
    ]
    return (
        colors[int(community) % len(colors)] if community is not None else "lightgray"
    )

In [26]:
def edge_to_source_community(edge):
    """Get the community of the source node of an edge."""
    source_node = next(
        (entry for entry in w.nodes if entry["properties"]["title"] == edge["start"]),
        None,
    )
    source_node_community = source_node["properties"]["community"]
    return source_node_community if source_node_community is not None else None

In [27]:
w.node_label_mapping = "title"

w.edge_thickness_factor_mapping = "weight"

In [28]:
w.circular_layout()

In [29]:
display(w)

GraphWidget(layout=Layout(height='800px', width='100%'))

In [30]:
w2 = GraphWidget()

w2.directed = True
w2.nodes = convert_entities_to_dicts(entity_df_crop)
w2.edges = convert_relationships_to_dicts(relationship_df_crop)

w2.node_label_mapping = "title"
# w2.edge_thickness_factor_mapping = "weight"
w2.organic_layout()

w2.node_color_mapping = lambda node: community_to_color(node["properties"]["type_c"])

display(w2)

GraphWidget(layout=Layout(height='800px', width='100%'))

## Граф контекста

In [31]:
from graphrag.config.enums import ModelType
from graphrag.config.models.language_model_config import LanguageModelConfig
from graphrag.language_model.manager import ModelManager

In [32]:
api_key = os.environ["GRAPHRAG_API_KEY"]

In [36]:
llm_model = "gpt-4o-mini"
embedding_model = "text-embedding-3-small"

In [37]:
chat_config = LanguageModelConfig(
    api_key=api_key,
    api_base="https://api.vsegpt.ru/v1",
    type=ModelType.OpenAIChat,
    model=llm_model,
    max_retries=20,
)
chat_model = ModelManager().get_or_create_chat_model(
    name="local_search",
    model_type=ModelType.OpenAIChat,
    config=chat_config,
)

token_encoder = tiktoken.encoding_for_model(llm_model)

embedding_config = LanguageModelConfig(
    api_key=api_key,
    api_base="https://api.vsegpt.ru/v1",
    type=ModelType.OpenAIEmbedding,
    model=embedding_model,
    max_retries=20,
)

text_embedder = ModelManager().get_or_create_embedding_model(
    name="local_search_embedding",
    model_type=ModelType.OpenAIEmbedding,
    config=embedding_config,
)

In [34]:
description_embedding_store = LanceDBVectorStore(collection_name="default-entity-description")
description_embedding_store.connect(db_uri=LANCEDB_URI)

entities = pd.read_parquet(f"{directory}/output/entities.parquet")
communities = pd.read_parquet(f"{directory}/output/communities.parquet")
community_reports = pd.read_parquet(f"{directory}/output/community_reports.parquet")
text_units = pd.read_parquet(f"{directory}/output/text_units.parquet")
relationships = pd.read_parquet(f"{directory}/output/relationships.parquet")

entities_d = read_indexer_entities(entity_df, communities, COMMUNITY_LEVEL)
relationships_d = read_indexer_relationships(relationships)
reports_d = read_indexer_reports(community_reports, communities, COMMUNITY_LEVEL)
text_units_d = read_indexer_text_units(text_units)

In [38]:
context_builder = LocalSearchMixedContext(
    community_reports=reports_d,
    text_units=text_units_d,
    entities=entities_d,
    relationships=relationships_d,
    # if you did not run covariates during indexing, set this to None
    covariates=None, #covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

In [39]:
local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False, #True,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

model_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [40]:
search_engine = LocalSearch(
    model=chat_model,
    context_builder=context_builder,
    token_encoder=token_encoder,
    model_params=model_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [49]:
start = time.time()

result = await search_engine.search("Как построить фаску?")

end = time.time()
print(f"Длительность: {(end - start):.3f} сек.")

Длительность: 16.623 сек.


In [50]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,1634,ФАСКА ПО УГЛУ,"Команда для создания фаски, где задается значе...",5,True
1,1854,РЕЖИМ «СИММЕТРИЧНАЯ ФАСКА»,"Режим работы команды «Фаска», при котором созд...",1,True
2,1633,ФАСКА ПО СМЕЩЕНИЯМ,"Команда для создания фаски, где значения смеще...",5,True
3,1853,ПОЛЕ «ОБЪЕКТЫ ДЛЯ ФАСКИ»,"Параметр, предназначенный для задания рёбер, к...",2,True
4,1852,КОМАНДА «ФАСКА»,"Команда, предназначенная для создания фаски на...",4,True


In [51]:
result.context_data["relationships"].head()

,id,source,target,description,weight,links,in_context
0,1901,СИММЕТРИЧНАЯ ФАСКА,ДИАЛОГ ПАРАМЕТРОВ КОМАНДЫ,В диалоге параметров команды выбираются линии ...,8.0,1,True
1,1905,ФАСКА ПО СМЕЩЕНИЯМ,ДИАЛОГ ПАРАМЕТРОВ КОМАНДЫ,В диалоге параметров команды выбираются линии ...,8.0,2,True
2,1909,ФАСКА ПО УГЛУ,ДИАЛОГ ПАРАМЕТРОВ КОМАНДЫ,В диалоге параметров команды выбираются линии ...,8.0,2,True
3,2232,КОМАНДА «ФАСКА»,РЕЖИМ «СИММЕТРИЧНАЯ ФАСКА»,Режим «Симметричная фаска» является одним из р...,8.0,1,True
4,2233,КОМАНДА «ФАСКА»,РЕЖИМ «ФАСКА ПО СМЕЩЕНИЯМ»,Режим «Фаска по смещениям» является другим реж...,1.0,1,True


In [52]:
result.context_data["entities"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       19 non-null     object
 1   entity                   19 non-null     object
 2   description              19 non-null     object
 3   number of relationships  19 non-null     object
 4   in_context               19 non-null     bool  
dtypes: bool(1), object(4)
memory usage: 759.0+ bytes


In [53]:
result.context_data["relationships"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           44 non-null     object
 1   source       44 non-null     object
 2   target       44 non-null     object
 3   description  44 non-null     object
 4   weight       44 non-null     object
 5   links        44 non-null     object
 6   in_context   44 non-null     bool  
dtypes: bool(1), object(6)
memory usage: 2.2+ KB


In [54]:
def show_graph(result):

    if (
        "entities" not in result.context_data
        or "relationships" not in result.context_data
    ):
        msg = "The passed results do not contain 'entities' or 'relationships'"
        raise ValueError(msg)

    # converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_entities_to_dicts(df):
        """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
        nodes_dict = {}
        for _, row in df.iterrows():
            # Create a dictionary for each row and collect unique nodes
            node_id = row["entity"]
            if node_id not in nodes_dict:
                nodes_dict[node_id] = {
                    "id": node_id,
                    "properties": row.to_dict(),
                }
        return list(nodes_dict.values())

    # converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_relationships_to_dicts(df):
        """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
        relationships = []
        for _, row in df.iterrows():
            # Create a dictionary for each row
            relationships.append({
                "start": row["source"],
                "end": row["target"],
                "properties": row.to_dict(),
            })
        return relationships

    w = GraphWidget()
    # use the converted data to visualize the graph
    w.nodes = convert_entities_to_dicts(result.context_data["entities"])
    w.edges = convert_relationships_to_dicts(result.context_data["relationships"])
    w.directed = True
    # show title on the node
    w.node_label_mapping = "entity"
    # use weight for edge thickness
    w.edge_thickness_factor_mapping = "weight"
    display(w)

In [55]:
show_graph(result)

GraphWidget(layout=Layout(height='690px', width='100%'))

### Расширенный граф контекста

In [48]:
import graphrag.api as api
from graphrag.config.load_config import load_config
from graphrag.index.typing.pipeline_run_result import PipelineRunResult

import os

os.getcwd()

'C:\\Users\\glvv2\\hr_assistant'

In [56]:
directory = os.path.join(os.getcwd(), "graphragcad")
directory

'C:\\Users\\glvv2\\hr_assistant\\graphragcad'

In [57]:
graphrag_config = load_config(Path(directory))

entities = pd.read_parquet(f"{directory}/output/entities.parquet")
communities = pd.read_parquet(f"{directory}/output/communities.parquet")
community_reports = pd.read_parquet(f"{directory}/output/community_reports.parquet")
text_units = pd.read_parquet(f"{directory}/output/text_units.parquet")
relationships = pd.read_parquet(f"{directory}/output/relationships.parquet")

In [58]:
question = "Как построить фаску?"

In [59]:
start = time.time()

response, context = await api.local_search(
    config=graphrag_config,
    entities=entities,
    communities=communities,
    community_reports=community_reports,
    text_units=text_units,
    relationships=relationships,
    covariates=None,
    community_level=2,
    response_type="Multiple Paragraphs",
    query=question,
)

end = time.time()


INFO: Vector Store Args: {
    "default_vector_store": {
        "type": "lancedb",
        "db_uri": "C:\\Users\\glvv2\\hr_assistant\\graphragcad\\output\\lancedb",
        "url": null,
        "audience": null,
        "container_name": "==== REDACTED ====",
        "database_name": null,
        "overwrite": true
    }
}


In [60]:
print(f"Длительность: {(end - start):.3f} сек.")

Длительность: 18.204 сек.


In [61]:
print(response)

## Построение фаски в CAD

Построение фаски в CAD (компьютерном проектировании) является важной задачей, которая позволяет создавать гладкие переходы между рёбрами объектов. В CAD программном обеспечении предусмотрены различные режимы для создания фасок, каждый из которых имеет свои особенности и параметры.

### Основные режимы фаски

1. **Симметричная фаска**: Этот режим позволяет создать равнобокую фаску, где задается одно значение смещения, применяемое ко всем рёбрам. Для активации этого режима необходимо выбрать соответствующий способ в диалоге параметров команды [Data: Sources (171, 198)].

2. **Фаска по смещениям**: В этом режиме пользователи могут задавать разные значения смещения для каждой линии, что позволяет более точно контролировать форму фаски. Поля для задания смещений активируются после выбора рёбер в диалоге параметров [Data: Sources (171, 198)].

3. **Фаска по углу**: Этот режим требует задания значения смещения вдоль первой линии и угла от неё. Это позволяет создават

In [62]:
context.keys()

dict_keys(['reports', 'relationships', 'claims', 'entities', 'sources'])

In [64]:
context["entities"].head()

,id,entity,description,number of relationships,in_context
0,1634,ФАСКА ПО УГЛУ,"Команда для создания фаски, где задается значе...",5,True
1,1854,РЕЖИМ «СИММЕТРИЧНАЯ ФАСКА»,"Режим работы команды «Фаска», при котором созд...",1,True
2,1633,ФАСКА ПО СМЕЩЕНИЯМ,"Команда для создания фаски, где значения смеще...",5,True
3,1853,ПОЛЕ «ОБЪЕКТЫ ДЛЯ ФАСКИ»,"Параметр, предназначенный для задания рёбер, к...",2,True
4,1852,КОМАНДА «ФАСКА»,"Команда, предназначенная для создания фаски на...",4,True


In [65]:
context["relationships"].head()

,id,source,target,description,weight,links,in_context
0,1901,СИММЕТРИЧНАЯ ФАСКА,ДИАЛОГ ПАРАМЕТРОВ КОМАНДЫ,В диалоге параметров команды выбираются линии ...,8.0,1,True
1,1905,ФАСКА ПО СМЕЩЕНИЯМ,ДИАЛОГ ПАРАМЕТРОВ КОМАНДЫ,В диалоге параметров команды выбираются линии ...,8.0,2,True
2,1909,ФАСКА ПО УГЛУ,ДИАЛОГ ПАРАМЕТРОВ КОМАНДЫ,В диалоге параметров команды выбираются линии ...,8.0,2,True
3,2232,КОМАНДА «ФАСКА»,РЕЖИМ «СИММЕТРИЧНАЯ ФАСКА»,Режим «Симметричная фаска» является одним из р...,8.0,1,True
4,2233,КОМАНДА «ФАСКА»,РЕЖИМ «ФАСКА ПО СМЕЩЕНИЯМ»,Режим «Фаска по смещениям» является другим реж...,1.0,1,True


In [67]:
context["entities"].shape, context["relationships"].shape

((19, 5), (44, 7))

In [68]:
w_ans = GraphWidget()
w_ans.directed = True

In [69]:
def convert_entities_to_dicts_ans_full_2(df):
    """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
    nodes_dict = {}
    for _, row in df.iterrows():
        # Create a dictionary for each row and collect unique nodes
        # node_id = df_ent.loc[df_ent["title"] == row["source"], "human_readable_id"].values[0]
        node_id = row["source"]
        # print(node_id)
        if node_id not in nodes_dict:
            nodes_dict[node_id] = {
                "id": node_id,
                "properties": row.to_dict(),
            }
            nodes_dict[node_id]["properties"]["label"] = row["source"]
        # node_id = df_ent.loc[df_ent["title"] == row["target"], "human_readable_id"].values[0]
        node_id = row["target"]
        # print(node_id)
        if node_id not in nodes_dict:
            nodes_dict[node_id] = {
                "id": node_id,
                "properties": row.to_dict(),
            }
            nodes_dict[node_id]["properties"]["label"] = row["target"]
    return list(nodes_dict.values())

In [70]:
w_ans.nodes = convert_entities_to_dicts_ans_full_2(context["relationships"])

In [71]:
def convert_relationships_to_dicts(df):
    """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
    relationships = []
    for _, row in df.iterrows():
        # Create a dictionary for each row
        relationships.append({
            "start": row["source"],
            "end": row["target"],
            "properties": row.to_dict(),
        })
    return relationships

In [72]:
w_ans.edges = convert_relationships_to_dicts(context["relationships"])
w_ans.edge_thickness_factor_mapping = "weight"
w_ans.organic_layout()

In [73]:
display(w_ans)

GraphWidget(layout=Layout(height='800px', width='100%'))